In [12]:
import zktool as zk
import numpy as np
import sys
from numpy import poly1d
from scipy.interpolate import lagrange
import galois
from functools import reduce

#On definit le corps de Galois
p=21888242871839275222246405745257275088548364400416034343698204186575808495617
GF = galois.GF(p) 

In [13]:
eq_input=input("Enter the equation: de type x^3 + x + 5") #METTRE DES COEFFICIENTS >=0, aucune negatif pour l'instant

In [14]:
def transformer_equation(enonce):
    lignes = enonce.split('\n')
    lignes_nettoyees = [ligne.strip() for ligne in lignes if ligne.strip()]
    return lignes_nettoyees



#Les équations va etre automatique determiné avec votre input (Polynome)
equations = zk.decompose_polynomial(eq_input)

#Voici une méthode rentrer les équations manuellements:

# Pour tester à la main il faut juste copier coller ses équations dans la variable manual_equation en dessus. Sinon laisser vide (manual_equation="").
manual_equation='''
v1 = x * x
v2 = x * v1
v3 = 3 * v2
v4 = 5 * v1
v5 = 10 * x
out = v3 + v4 + v5 + 3
'''
manual_equation=""

if manual_equation:
    equations=transformer_equation(manual_equation)

unique_words = zk.get_unique_words(equations)


print("Modelisation des circuits arithmetiques :")
for eq in equations:
    print(eq)

print("\nEnsemble des varibles =",unique_words)


Modelisation des circuits arithmetiques :
v1 = x * x
v2 = v1 * x
v3 = v2 + x
out = v3 + 5

Ensemble des varibles = ['1', 'out', 'x', 'v1', 'v2', 'v3']


In [4]:
#création des 3 matrice vides A B C
L= []
R= []
O= []

print("Equations :")
for eq in equations:
    print(eq)
print(" ")

ref_array=list(unique_words)
print("Vecteur solution :")
print(ref_array)
print(' ')

print("Matrice L R O :")
for eq in equations:
    eq_split=eq.split()

    if eq_split[3]=='*':
        vecta=zk.get_position_vector(ref_array, eq_split[2])
        L.append(vecta)
        vectb=zk.get_position_vector(ref_array, eq_split[4])
        R.append(vectb)
        vectc=zk.get_position_vector(ref_array, eq_split[0])
        O.append(vectc)

    if eq_split[3]=='+':
        if "out" in eq:
            list_var = eq_split[2:]
        else:
            list_var = [eq_split[2],eq_split[4]]
        vecta=zk.get_position_vector(ref_array, list_var)
        L.append(vecta)
        vectb=zk.get_position_vector(ref_array, "1")
        R.append(vectb)
        vectc=zk.get_position_vector(ref_array, eq_split[0])
        O.append(vectc)

print("L=")
for ligne in L:
    print(ligne)
print(" ")

print("R=")
for ligne in R:
    print(ligne)
print(" ")

print("O=")
for ligne in O:
    print(ligne)
print(" ")

Equations :
v1 = x * x
v2 = v1 * x
v3 = v2 + x
out = v3 + 5
 
Vecteur solution :
['1', 'out', 'x', 'v1', 'v2', 'v3']
 
Matrice L R O :
L=
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 1, 0]
[5, 0, 0, 0, 0, 1]
 
R=
[0, 0, 1, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
 
O=
[0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 1]
[0, 1, 0, 0, 0, 0]
 


In [5]:
def polynomial_product_in_GF(values, ordre):
    '''
    Fonction qui transforme une liste de valeurs en un polynome dans le corps de Galois GF
    ex: [1,2,3] = (x-1)(x-2)(x-3)
    Il return ce polynome dans le corps de Galois 'ordre'
    
    '''
    result_poly = galois.Poly([1], field=ordre)  # Start with the polynomial '1' in the given field
    for val in values:
        # Multiply the result by (x - val) for each val in values
        result_poly *= galois.Poly([1, -val], field=ordre)
    return result_poly

num_eq = len(L)
def interpolate_column(col):
    xs = GF(np.array([i for i in range(1, num_eq+1)]))
    return galois.lagrange_poly(xs, col)

def inner_product_polynomials_with_witness(polys, witness):
    mul_ = lambda x, y: x * y
    sum_ = lambda x, y: x + y
    return reduce(sum_, map(mul_, polys, witness))

In [6]:
#Ici on cherche les coefficients des polynomes de chaque matrice

L=np.array(L)
R=np.array(R)
O=np.array(O)


# On convertit les matrices pour etre convertisable en GF (Chaque valeur est ramenée entre [0, p-1] )
L = L % p
R = R % p
O = O % p
#On convertit les matrices en matrices de Galois
L_galois = GF(L)
R_galois = GF(R)
O_galois = GF(O)

# axis 0 is the columns. apply_along_axis is the same as doing a for loop over the columns and collecting the results in an array
U_polys = np.apply_along_axis(interpolate_column, 0, L_galois)
print("L_galois :\n", L_galois)
print("\nU_polys :")
print(U_polys[0])
print(" ")
for elem in U_polys:
    print(" ",elem)

V_polys = np.apply_along_axis(interpolate_column, 0, R_galois)
print("\n\nR_galois :\n", R_galois)
print("\nV_polys :")
for elem in V_polys:
    print(" ",elem)

W_polys = np.apply_along_axis(interpolate_column, 0, O_galois)
print("\n\nO_galois :\n", O_galois)
print("\nW_polys :")

for elem in W_polys:
    print(" ",elem)



print(type(U_polys[0]))

print(V_polys)
print(W_polys)

L_galois :
 [[0 0 1 0 0 0]
 [0 0 0 1 0 0]
 [0 0 1 0 1 0]
 [5 0 0 0 0 1]]

U_polys :
3648040478639879203707734290876212514758060733402672390616367364429301415937x^3 + 21888242871839275222246405745257275088548364400416034343698204186575808495612x^2 + 18240202393199396018538671454381062573790303667013361953081836822146507079690x + 21888242871839275222246405745257275088548364400416034343698204186575808495612
 
  3648040478639879203707734290876212514758060733402672390616367364429301415937x^3 + 21888242871839275222246405745257275088548364400416034343698204186575808495612x^2 + 18240202393199396018538671454381062573790303667013361953081836822146507079690x + 21888242871839275222246405745257275088548364400416034343698204186575808495612
  0
  14592161914559516814830937163504850059032242933610689562465469457717205663744x^3 + 5x^2 + 7296080957279758407415468581752425029516121466805344781232734728858602831861x + 8
  10944121435919637611123202872628637544274182200208017171849102093287904247809x^3 + 2

In [7]:
values = {"x": 5}

witness = zk.compute_solution_vector(ref_array, equations, values)
print("\nSolution Vector:", witness)
print("Vecteur avec variable ",ref_array)

#idem pour le vecteur witness
witness = np.array(witness) % p

num_var=len(witness)

v1 = 25
v2 = 125
v3 = 130
out = 135

Solution Vector: [1, 135, 5, 25, 125, 130]
Vecteur avec variable  ['1', 'out', 'x', 'v1', 'v2', 'v3']


In [8]:
#Conversion du witness dans le corps de Galois
witness = GF(witness)

fu = inner_product_polynomials_with_witness(U_polys, witness)
print("\nfu(x) = ", fu)
fv = inner_product_polynomials_with_witness(V_polys, witness)
print("fv(x) = ", fv)
fw = inner_product_polynomials_with_witness(W_polys, witness)
print("fw(x) = ", fw)


fu(x) =  18240202393199396018538671454381062573790303667013361953081836822146507079650x^3 + 10944121435919637611123202872628637544274182200208017171849102093287904248036x^2 + 14592161914559516814830937163504850059032242933610689562465469457717205663298x + 255
fv(x) =  14592161914559516814830937163504850059032242933610689562465469457717205663746x^3 + 21888242871839275222246405745257275088548364400416034343698204186575808495607x^2 + 7296080957279758407415468581752425029516121466805344781232734728858602831893x + 21888242871839275222246405745257275088548364400416034343698204186575808495610
fw(x) =  3648040478639879203707734290876212514758060733402672390616367364429301415952x^3 + 10944121435919637611123202872628637544274182200208017171849102093287904247666x^2 + 7296080957279758407415468581752425029516121466805344781232734728858602832289x + 21888242871839275222246405745257275088548364400416034343698204186575808495352


In [9]:
values = [i for i in range(1, num_eq+1)] # Car on a 6 colonnes

t = polynomial_product_in_GF(values, GF)

h_quo = (fu * fv - fw) // t
h_rem = (fu * fv - fw) % t

print("\nh_quo(x) = ", h_quo)
print("h_rem(x) = ", h_rem)
print("t(x) = ", t,"\n")
if(h_rem == 0):
    print("=> La preuve est valide")
else:
    print("=> La preuve invalide")


h_quo(x) =  2432026985759919469138489527250808343172040488935114927077578242952867610583x^2 + 9728107943039677876553958109003233372688161955740459708310312971811470442697x + 7296080957279758407415468581752425029516121466805344781232734728858602831809
h_rem(x) =  0
t(x) =  x^4 + 21888242871839275222246405745257275088548364400416034343698204186575808495607x^3 + 35x^2 + 21888242871839275222246405745257275088548364400416034343698204186575808495567x + 24 

=> La preuve est valide


# SUITE AVEC ECC

In [10]:
from py_ecc.optimized_bn128 import multiply, G1, G2, add, pairing, neg, normalize

tau = GF(20)

print(f"τ = {tau}")

T = galois.Poly([1, p-1], field=GF)
for i in range(2, L.shape[0] + 1):
    T *= galois.Poly([1, p-i], field=GF)

print("\nT = ", T)

T_tau = T(tau)
print(f"\nT(τ) = {T_tau}")

# G1[τ^0], G1[τ^1], ..., G1[τ^d]
tau_G1 = [multiply(G1, int(tau**i)) for i in range(0, T.degree)]
# G1[τ^0 * T(τ)], G1[τ^1 * T(τ)], ..., G1[τ^d-1 * T(τ)]
target_G1 = [multiply(G1, int(tau**i * T_tau)) for i in range(0, T.degree - 1)]

# G2[τ^0], G2[τ^1], ..., G2[τ^d-1]
tau_G2 = [multiply(G2, int(tau**i)) for i in range(0, T.degree)]

print("Trusted setup:")
print("-"*10)
print(f"[τ]G1 = {[normalize(point) for point in tau_G1]}")
print(f"[T(τ)]G1 = {[normalize(point) for point in target_G1]}")

print(f"\n[τ]G2 = {[normalize(point) for point in tau_G2]}")

τ = 20

T =  x^4 + 21888242871839275222246405745257275088548364400416034343698204186575808495607x^3 + 35x^2 + 21888242871839275222246405745257275088548364400416034343698204186575808495567x + 24

T(τ) = 93024
Trusted setup:
----------
[τ]G1 = [(1, 2), (18947110137775984544896515092961257947872750783784269176923414004072777296602, 12292085037693291586083644966434670280746730626861846747147579999202931064992), (16262199471205794413544947826745938654132104752637586692048329713311590397011, 13296900385261935021718889695689394625708483652039722230815936262285054528714), (21603600070689675766438470661345954782419355034652174505468210225883925863279, 15787091953565760722773063158476721787069408761080596737736006929439659337677)]
[T(τ)]G1 = [(3572442620270260577858775988072594023321363005596886626594361777205456902409, 6415150419557488235487659370376266763189375755418107343824154603017059521168), (14903192452168551099695688573387439557274106853172417561540365357828216003565, 5080667453105766792

In [11]:

U = galois.Poly((GF(ref_array) @ U_polys[0])[::-1])
V = galois.Poly((ref_array @ V_polys)[::-1])
W = galois.Poly((ref_array @ W_polys)[::-1])

print("U = ", U)
print("V = ", V)
print("W = ", W)

H = (U * V - W) // T
rem = (U * V - W) % T

print("H = ", H)
print("rem = ", rem)

# assert rem == 0

u = U(tau)
v = V(tau)
_w = W(tau)
ht = H(tau)*T_tau

# assert u * v - _w == ht, f"{u} * {v} - {_w} != {ht}"

ValueError: Found multiple polynomial indeterminates <built-in function vars> in string 'out'.